In [77]:
%load_ext autoreload
%autoreload 2

In [78]:
from Datos import Datos
import EstrategiaParticionado
import Clasificador

import numpy as np
import random

from sklearn.preprocessing import OneHotEncoder

# <font color='red'>Entrenamiento y clasificación con Clasificador.py</font>


## <font color='blue'> Dataset tic-tac-toe </font>

In [79]:
dataset_ttt = Datos("../ConjuntosDatos/tic-tac-toe.data")
ndata, ncols = dataset_ttt.datos.shape # nfeat = ncols-1 
print("Numero de ejemplos de entrenamiento:", ndata, ", Numero de columnas:", ncols)

Numero de ejemplos de entrenamiento: 958 , Numero de columnas: 10


In [80]:
np.random.shuffle(dataset_ttt.datos)

NB = Clasificador.ClasificadorNaiveBayes()

### Validación simple

In [81]:
strat_simple = EstrategiaParticionado.ValidacionSimple(0.8)

In [82]:
NB.validacion(strat_simple, dataset_ttt, NB)

[0.234375]

### Validación simple con varias repeticiones (3 reps)

In [83]:
strat_simple_rep3 = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)

In [84]:
errores = NB.validacion(strat_simple_rep3, dataset_ttt, NB)
print("Errores:", errores)
print("Error medio:", np.mean(errores), "+-", np.std(errores))

Errores: [0.3125, 0.30208333333333337, 0.27083333333333337]
Error medio: 0.2951388888888889 +- 0.017704928866641604


### Validación Cruzada (5 folds)

In [85]:
strat_cross = EstrategiaParticionado.ValidacionCruzada(k_fold=5)

In [86]:
errores = NB.validacion(strat_cross, dataset_ttt, NB)
print("Errores:", errores)
print("Error medio:", np.mean(errores), "+-", np.std(errores))

Errores: [0.2984293193717278, 0.24607329842931935, 0.37696335078534027, 0.2879581151832461, 0.32460732984293195]
Error medio: 0.3068062827225131 +- 0.04325000299070864


## <font color='blue'> Dataset German </font>

In [87]:
dataset_ger = Datos("../ConjuntosDatos/german.data")
ndata, ncols = dataset_ger.datos.shape # nfeat = ncols-1 
print("Numero de ejemplos de entrenamiento:", ndata, ", Numero de columnas:", ncols)

Numero de ejemplos de entrenamiento: 1000 , Numero de columnas: 21


In [88]:
np.random.shuffle(dataset_ger.datos)

### Validación simple

In [89]:
strat_simple_ger = EstrategiaParticionado.ValidacionSimple(0.8)
NB.validacion(strat_simple_ger, dataset_ger, NB)

[0.275]

### Validación simple con varias repeticiones (3 reps)

In [90]:
strat_simple_rep3_ger = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)
NB.validacion(strat_simple_rep3_ger, dataset_ger, NB)

[0.28, 0.21999999999999997, 0.28500000000000003]

### Validación cruzada (5 folds)

In [91]:
strat_cross_ger = EstrategiaParticionado.ValidacionCruzada(k_fold=5)
errores = NB.validacion(strat_cross_ger, dataset_ger, NB)
print("Errores:", errores)
print("Error medio:", np.mean(errores), "+-", np.std(errores))

Errores: [0.24, 0.245, 0.26, 0.28, 0.28]
Error medio: 0.261 +- 0.01685229954635273


# <font color='red'>Entrenamiento y clasificación con funciones de sklearn (encapsuladas en Clasificador.py)</font>

In [92]:
# Las funciones de sklearn requieren una codificación de los atributos discretos diferente a la nuestra.
# Por lo tanto, realizamos un pequeño trabajo de preprocesado de datos para que sus funciones de sklearn
# puiedan realizar un entrenamiento y clasificación equiparable al nuestro

def preprocess_data(dataset):
    X = dataset.datos[:,:-1] # all rows, all columns but last one
    Y = dataset.datos[:,-1]  # all rows, just last column (class)
    
    # One hot encoding for discrete features
    enc = OneHotEncoder(sparse=False, categories='auto')
    X_enc = np.array(enc.fit_transform(X)) 
    # Concatenating encoded data matrix and classes
    datos_aux = np.concatenate((X_enc, Y[:,None]), axis=1)
    # Updating dataset
    dataset.datos = np.zeros(datos_aux.shape)
    dataset.datos = datos_aux
    return dataset 

## <font color='blue'> Dataset tic-tac-toe </font>

In [93]:
ttt_db = Datos('../ConjuntosDatos/tic-tac-toe.data')
ndata, ncols = ttt_db.datos.shape # nfeat = ncols-1 
print("Numero de ejemplos de entrenamiento:", ndata, ", Numero de columnas:", ncols)

Numero de ejemplos de entrenamiento: 958 , Numero de columnas: 10


In [94]:
ttt_db = preprocess_data(ttt_db)
np.random.shuffle(ttt_db.datos)

NBSK = Clasificador.ClasificadorNaiveBayesSK(gaussian_feat=False)

### Validación simple

In [95]:
strat_simple_sk = EstrategiaParticionado.ValidacionSimple(0.8)

errs = NBSK.validacion(strat_simple_sk, ttt_db, NBSK)

print(errs)

[0.29166666666666663]


### Validación simple con varias repeticiones (3 reps)

In [96]:
strat_simple_rep3_sk = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)

errs = NBSK.validacion(strat_simple_rep3_sk, ttt_db, NBSK)

print("Errores: ", errs)
print("Error medio:", np.mean(errs), "+-", np.std(errs))

Errores:  [0.30208333333333337, 0.32291666666666663, 0.27604166666666663]
Error medio: 0.3003472222222222 +- 0.019175973988172328


### Validacion cruzada (5 folds)

In [97]:
strat_cross_sk = EstrategiaParticionado.ValidacionCruzada(k_fold=5)

errs = NBSK.validacion(strat_cross_sk, ttt_db, NBSK)

print("Errores: ", errs)
print("Error medio:", np.mean(errs), "+-", np.std(errs))

Errores:  [0.3036649214659686, 0.2931937172774869, 0.31413612565445026, 0.3036649214659686, 0.2931937172774869]
Error medio: 0.3015706806282722 +- 0.007835931700050147


## <font color='blue'> Dataset German (todos los atributos se consideran discretos)</font>

In [98]:
ger_db = Datos('../ConjuntosDatos/german.data')
ndata, ncols = ger_db.datos.shape # nfeat = ncols-1 
print("Numero de ejemplos de entrenamiento:", ndata, ", Numero de columnas:", ncols)

Numero de ejemplos de entrenamiento: 1000 , Numero de columnas: 21


In [99]:
ger_db = preprocess_data(ger_db)
np.random.shuffle(ger_db.datos)

NBSK = Clasificador.ClasificadorNaiveBayesSK(gaussian_feat=False)

### Validación simple

In [100]:
strat_simple_sk = EstrategiaParticionado.ValidacionSimple(0.8)

errs = NBSK.validacion(strat_simple_sk, ger_db, NBSK)

print(errs)

[0.30000000000000004]


### Validación simple con varias repeticiones (3 reps)

In [101]:
strat_simple_rep3_sk = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)

errs = NBSK.validacion(strat_simple_rep3_sk, ger_db, NBSK)

print("Errores: ", errs)
print("Error medio:", np.mean(errs), "+-", np.std(errs))

Errores:  [0.30000000000000004, 0.245, 0.22499999999999998]
Error medio: 0.25666666666666665 +- 0.031710495984067444


### Validación cruzada (5 folds)

In [102]:
strat_cross_sk = EstrategiaParticionado.ValidacionCruzada(k_fold=5)

errs = NBSK.validacion(strat_cross_sk, ger_db, NBSK)

print("Errores: ", errs)
print("Error medio:", np.mean(errs), "+-", np.std(errs))

Errores:  [0.25, 0.30500000000000005, 0.19499999999999995, 0.31999999999999995, 0.29500000000000004]
Error medio: 0.27299999999999996 +- 0.04545327270945406


## <font color='blue'> Dataset German (todos los atributos se consideran continuos)</font>

In [103]:
ger_db_cont = Datos('../ConjuntosDatos/german.data')
ndata, ncols = ger_db_cont.datos.shape # nfeat = ncols-1 
print("Numero de ejemplos de entrenamiento:", ndata, ", Numero de columnas:", ncols)

Numero de ejemplos de entrenamiento: 1000 , Numero de columnas: 21


In [104]:
np.random.shuffle(ger_db.datos)

NBSK = Clasificador.ClasificadorNaiveBayesSK(gaussian_feat=True)

### Validación simple

In [105]:
strat_simple_sk = EstrategiaParticionado.ValidacionSimple(0.8)

errs = NBSK.validacion(strat_simple_sk, ger_db_cont, NBSK)

print(errs)

[0.26]


### Validación simple con varias repeticiones (3 reps)

In [106]:
strat_simple_rep3_sk = EstrategiaParticionado.ValidacionSimple(0.8, nreps=3)

errs = NBSK.validacion(strat_simple_rep3_sk, ger_db_cont, NBSK)

print("Errores: ", errs)
print("Error medio:", np.mean(errs), "+-", np.std(errs))

Errores:  [0.255, 0.22499999999999998, 0.275]
Error medio: 0.25166666666666665 +- 0.020548046676563275


### Validación cruzada (5 folds)

In [107]:
strat_cross_sk = EstrategiaParticionado.ValidacionCruzada(k_fold=5)

errs = NBSK.validacion(strat_cross_sk, ger_db_cont, NBSK)

print("Errores: ", errs)
print("Error medio:", np.mean(errs), "+-", np.std(errs))

Errores:  [0.29000000000000004, 0.27, 0.24, 0.27, 0.28]
Error medio: 0.27 +- 0.016733200530681527
